# Recomendador de artigos - OASIS

## Importação das bibliotecas

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.request import URLError, HTTPError
import pandas as pd

pd.options.display.max_columns = 999
pd.options.display.max_info_rows = 999

## Funções

In [4]:
def resposta_html(url,headers):
    '''
    Esta função recebe uma URL como entrada, faz uma solicitação HTTP usando o cabeçalho armazenado na variável de instância headers, 
    lê a resposta HTTP e retorna o HTML da resposta.
    '''
    try: 
        req = Request(url, headers= headers)
        response = urlopen(req)
        html = response.read()
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:
        print(e.reason)
    return html


def obtem_total_paginas(html):
    '''
    Esta função recebe o HTML da página de resultados de pesquisa e retorna o número total de páginas 
    de resultados que correspondem à pesquisa. 
    '''
    soup = BeautifulSoup(html, 'html.parser')
    total_paginas = soup.find('ul', {'class': 'pagination'}).find_all('li')[-1].get_text()
    total_paginas = total_paginas.replace('[', '').replace(']', '')
    total_paginas = int(total_paginas)
    return total_paginas


def links_paginas(url, total_paginas,headers):
    '''
    Esta função recebe uma URL de pesquisa e o número total de páginas de resultados como entrada. 
    Em seguida, extrai os links para todas as páginas de resultados e retorna uma lista de links para cada página.
    '''
    links_paginas = []
    numero_pagina = 1
    while numero_pagina < total_paginas + 1:
        numero_pagina = str(numero_pagina)
        resposta_html_pagina = resposta_html(url + f'&page={numero_pagina}',headers=headers)
        soup = BeautifulSoup(resposta_html_pagina, 'html.parser')
        for card in soup.findAll('div', {'class': 'result'}):
            links_paginas.append('https://oasisbr.ibict.br' + card.find('a', {'class': 'title getFull'}).get('href'))
        numero_pagina = int(numero_pagina)
        numero_pagina += 1
    return links_paginas


def scraping_dados(links_paginas,headers):
    '''
    A função extrai informações como título, autor, data de publicação e tipo de documento para cada página e armazena 
    as informações em uma lista.
    '''
    dados_lista = []
    
    for link in links_paginas:
        url = link
        html = resposta_html(url,headers=headers)
        soup = BeautifulSoup(html,'html.parser')

        table = soup.find('table',{'class':'citation table table-striped'})

        data = {}
        for row in table.find_all('tr'):
            th = row.find('th').get_text()
            td = row.find('td').get_text()
            data[th] = td
        dados_lista.append(data)


            

    return dados_lista


def dataframe_dados_lista(lista_dados):
    '''
    Esta função transforma uma lista de dicionários com os dados coletados da raspagem e retorna um DataFrame do pandas com esses dados.
    
    '''
    
    df = pd.DataFrame(lista_dados)[['title','description','url','eu_rights_str_mv','network_name_str','topic']]

    df['title'] = df['title'].str.strip().str.replace('\n                  ','')
    df['description'] = df['description'].str.strip().str.replace('\n                  ','')
    df['url'] = df['url'].str.replace('\n                  ','').str.replace('\n','')
    df['eu_rights_str_mv'] = df['eu_rights_str_mv'].str.replace('\n                  ','').str.replace('\n','')
    df['network_name_str'] = df['network_name_str'].str.replace('\n                  ','').str.replace('\n','')
    df['topic'] = df['topic'].str.strip().str.replace('.','').str.split('\n                  ').apply(lambda x: ';'.join(x) if isinstance(x, list) else x)

    return df

## Coleta de dados

In [6]:
#url = 'https://oasisbr.ibict.br/vufind/Search/Results?filter%5B%5D=format%3A%22article%22&type=AllFields&daterange%5B%5D=publishDate&publishDatefrom=2023&publishDateto=2023'
#headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

#resposta = resposta_html(url,headers)

#total_paginas = obtem_total_paginas(resposta)

#links = links_paginas(url,total_paginas,headers)

In [7]:
#lista_dados = scraping_dados(links,headers)

#df = dataframe_dados_lista(lista_dados)

#df.to_csv('artigos_oasis_2023_20231003.csv',sep=';',index=False)


## Transformações dos dados

In [130]:
df = pd.read_csv('artigos_oasis_2023_20231003.csv',sep=';')

In [131]:
df['url'] = df['url'].str.replace('\n','')

In [137]:
df['qtd_termos'] = df['topic'].str.split(';').apply(lambda x: len(x) if isinstance(x,list) else 0)

In [139]:
df.qtd_termos.value_counts()

qtd_termos
1      3189
5      3052
0      2184
4      2101
6      1957
8      1022
3       998
7       785
10      733
9       701
12      447
2       322
15      241
11      223
13      103
14      103
16       76
20       41
18       34
17       22
19       11
21        9
22        6
25        4
23        3
28        3
30        2
46        1
24        1
29        1
27        1
109       1
Name: count, dtype: int64

In [138]:
df.qtd_termos.value_counts(normalize=True) * 100

qtd_termos
1      17.353213
5      16.607716
0      11.884421
4      11.432769
6      10.649181
8       5.561299
3       5.430701
7       4.271644
10      3.988682
9       3.814551
12      2.432388
2       1.752190
15      1.311422
11      1.213473
13      0.560483
14      0.560483
16      0.413560
20      0.223105
18      0.185014
17      0.119715
19      0.059857
21      0.048974
22      0.032650
25      0.021766
23      0.016325
28      0.016325
30      0.010883
46      0.005442
24      0.005442
29      0.005442
27      0.005442
109     0.005442
Name: proportion, dtype: float64

In [151]:
(df.qtd_termos.value_counts(normalize=True) * 100).reset_index().query('3 < qtd_termos < 10').proportion.sum()

52.337160581161235

In [152]:
df_filtrado = df.query('3 < qtd_termos < 10')

In [154]:
#Verificar termos do "topic"
for i in df_filtrado['topic']:
    print(i)

Ciência da implementação;Saúde mental;Políticas de saúde;Estratégia saúde da família;Atenção primária à saúde
Bioethics;Social sciences;Technology;Global health;Pharmaceutical industries
Educação médica;Povos indígenas;Programa Mais Médicos;Covid-19
Ativismo político;Recusa do paciente ao tratamento;Tratamento psiquiátrico involuntário;Saúde mental;Psiquiatria
biofloc technology;flocponics;halophytes;Litopenaeus vannamei;marine aquaponics
cuttings;essential oil;germination;gibberellic acid;indole-3-butyric acid;potassium nitrate
gluten;immunohistochemistry;LH;ovarian;transglutaminase
cell seeding;decellularized Wharton's jelly matrix;natural scaffolds;tissue decellularization;Wharton's jelly matrix
COVID-19;Fuzzy model;population dynamics;measure interventions
esat-6;molecular identification;nontuberculous mycobacteria;tuberculosis;Mycobacterium avium
Trifluralin;Aloe vera;life span;DNA damage;larval toxicity
bioprospection;DNA-barcode;rbcL;Navicula pseudoantonii;Nitzschia inconspicua


In [156]:
#Verificar termos do "topic"
for i in df_filtrado['topic']:
    if ',' in i:
        print(i)

Tooth abnormalities;Prevalence;Adolescents;Esthetics, dental
Breast feeding;Breast-milk substitutes;Direct-to-consumer advertising;Legislation, food;Products commerce
Animals, poisonous;Public health surveillance;Epidemiological monitoring;Incidence;Accidents
Preeclampsia;Infant, premature;Gastrointestinal tract;Enteral feeding
Neonatal sepsis;Mortality;Morbidity;Infant, Very low birth weight;Infant, extremely premature
Caregivers;Sleep;Stress, psychological;Nursing;Cleft lip;Cleft palate
Calcium, dietary;Child;Brazil;Dairy products
Drugs utilization;Infant, newborn;Pharmaceutical preparations;Off-label use;Intensive care units, neonatal
Dermatitis, allergic contact;Dermatitis, irritant;Play and playthings;Patch tests;Child
Pregnancy;Syphilis;Syphilis, Congenital;Child Care;Symbolic Interactionism
Pessoas com surdez;Identificação Social;Estudos de Validação;Persons with hearing impairments;Social Identification;Validation Study;Personas con Deficiencia Auditiva, Identificación Social, 